In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns

from ray import init, rllib, shutdown

In [ ]:
from attack_simulator.agents import ATTACKERS
from attack_simulator.env import AttackSimulationEnv
from attack_simulator.graph import AttackGraph, SIZES

In [ ]:
class NoAction(rllib.policy.Policy):
    def compute_actions(self, observations, *args, **kwargs):
        # FIXME: use a `numpy` array as a temporary workaround for
        #        https://github.com/ray-project/ray/issues/10100
        return np.zeros(len(observations)), [], {}

    def get_weights(self):
        return {}

    def set_weights(self, weights):
        pass


no_action = rllib.agents.trainer_template.build_trainer(name="NoAction", default_policy=NoAction)

In [ ]:
from ray.util.client import worker

worker.INITIAL_TIMEOUT_SEC = worker.MAX_TIMEOUT_SEC = 1


def ray_init():
    if os.path.isdir("/var/run/secrets/kubernetes.io") or os.path.exists(
        os.path.expanduser("~/ray_bootstrap_config.yaml")
    ):
        # inside k8s pod or a cluster managed by Ray's autoscaler
        context = init(address="auto")
    else:
        ray_client_server = "host.docker.internal" if os.path.exists("/.dockerenv") else "127.0.0.1"
        try:
            context = init(address=f"ray://{ray_client_server}:10001")
        except ConnectionError:
            # clean up after failed connection attempt
            shutdown()
            # listen on all interfaces inside a container for port-forwarding to work
            dashboard_host = "0.0.0.0" if os.path.exists("/.dockerenv") else "127.0.0.1"
            context = init(num_cpus=4, dashboard_host=dashboard_host)
    print("\x1b[33;1m", context, "\x1b[m")
    return context

In [ ]:
from tqdm.auto import tqdm

rename = dict(
    attacker="Attacker",
    graph_size="Graph size",
    episode_reward="Returns",
    episode_lengths="Episode lengths",
)

num_episodes = 5  # run 5 episodes on the same environment
attackers = list(ATTACKERS)
graphs = [AttackGraph(dict(graph_size=size)) for size in SIZES]
seeds = [0, 1, 2, 3, 6, 7, 11, 28, 42, 1337]


def generate(savename):
    ray_init()

    frames = []
    for seed in tqdm(seeds, "seeds"):
        for attacker in tqdm(attackers, f"└── {seed}"):
            for graph in tqdm(graphs, f"\u00a0\u2001\u2001\u2001└── {attacker}@{seed}"):
                config = dict(
                    log_level="DEBUG",
                    framework="torch",
                    env=AttackSimulationEnv,
                    env_config=dict(attack_graph=graph, attacker=attacker),
                    seed=seed,
                    num_workers=0,
                    rollout_fragment_length=0,
                    evaluation_interval=1,
                    evaluation_num_workers=1,
                    evaluation_config=dict(explore=False),
                    evaluation_num_episodes=num_episodes,
                )
                agent = no_action(config=config)
                try:
                    stats = agent.evaluate()["evaluation"]["hist_stats"]
                    frame = pd.DataFrame(
                        dict(attacker=attacker, graph_size=graph.num_attacks, **stats)
                    )
                    frames.append(frame)
                except Exception as e:
                    print(seed, attacker, graph.graph_size, e)
                agent.stop()

    shutdown()
    results_df = pd.concat(frames, ignore_index=True).rename(columns=rename)
    results_df.to_csv(savename)
    return results_df

In [ ]:
savename = "length-agent-eval.csv"

df = generate(savename) if not os.path.exists(savename) else pd.read_csv(savename, index_col=0)

In [ ]:
df

In [ ]:
sns.set(style="darkgrid", rc={"figure.figsize": (12, 8)})

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Returns", hue="Attacker", ci="sd")
g.legend(title="Attacker", loc="lower left")
g.set_title("Defender: no-action")

In [ ]:
g = sns.lineplot(data=df, x="Graph size", y="Episode lengths", hue="Attacker", ci="sd")
g.legend(title="Attacker", loc="upper left")
g.set_title("Defender: no-action")

In [ ]:
pd.set_option("display.max_columns", 32)
df.groupby("Attacker").describe()

In [ ]:
import ray

ray.__version__

In [ ]:
help(ray.init)